### • 처음 크롤링 시 캡챠(보안문자) 등장 -> 보안문자 한번 넣은 다음 다시 크롤링하면 캡챠 안보임

In [1]:
#! pip3 install selenium
#! pip3 install webdriver-manager
#! pip3 install ipykernel

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
import re
import time

import itertools
from selenium.webdriver import ActionChains
#from tqdm import trange, notebook
from tqdm.notebook import tqdm

#로봇우회
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--ignore-certificate-errors')

#driver = webdriver.Chrome(options=chrome_options)

# 매크로 탐지 회피
user_agent=f'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
chrome_options.add_argument(user_agent)
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging', 'enable-automation'])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

driver = webdriver.Chrome(options=chrome_options)

# 웹페이지 해당 주소 이동
url = "https://omoney.kbstar.com/quics?page=C016559"
driver.get(url)

### 여러 페이지 크롤링

In [30]:
result = []

for i in tqdm(range(84, 171), desc='outer', position=0): #12

    table = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[3]/table')
    tbody = table.find_element(By.TAG_NAME, 'tbody')
    rows = tbody.find_elements(By.TAG_NAME, 'tr')

    for li in tqdm(range(len(rows)), desc='inner', position=1, leave=False):
        try:
            num = rows[li].find_element(By.CLASS_NAME, 'num').text
            title = rows[li].find_element(By.CLASS_NAME, 'left').text
            period = rows[li].find_elements(By.TAG_NAME, 'td')[2].get_attribute("innerText")
            count = rows[li].find_elements(By.TAG_NAME, 'td')[3].get_attribute("innerText")
            
            try:
                #click
                element = rows[li].find_element(By.CSS_SELECTOR, 'a')
                driver.execute_script("arguments[0].click();", element)
                time.sleep(0.7)
        
                image_url_1 = driver.find_element(By.CSS_SELECTOR, 'div.ev_visual > img').get_attribute('src')
                note = driver.find_element(By.CLASS_NAME, 'ev_tip_box').text

                x = [image_url_1]

                #x : 여러 이미지_url이 담긴 리스트
                try:
                    img_loc = driver.find_elements(By.CSS_SELECTOR, 'div.ev_Cont > div > img')
                    if img_loc:  # img_loc이 비어있지 않은지 확인
                        for j in range(len(img_loc)):
                            globals()[f'img_{j+2}'] = img_loc[j].get_attribute('src')
                            x.append(globals()[f'img_{j+2}'])

                    else:
                        raise Exception("No elements found in first selector")

                except:
                    try:
                        img_loc = driver.find_elements(By.CSS_SELECTOR, 'div.ev_Cont > div > p > img')
                        if img_loc:  # img_loc이 비어있지 않은지 확인
                            for j in range(len(img_loc)):
                                globals()[f'img_{j+2}'] = img_loc[j].get_attribute('src')
                                x.append(globals()[f'img_{j+2}'])
                        else:
                            raise Exception("No elements found in second selector")

                    except:
                        img_loc = driver.find_elements(By.CSS_SELECTOR, 'div.ev_Cont > div > div > img')
                        if img_loc:  
                            for j in range(len(img_loc)):
                                globals()[f'img_{j+2}'] = img_loc[j].get_attribute('src')
                                x.append(globals()[f'img_{j+2}'])
                        else:
                            print("No elements found in third selector")
                

                data = [num, title, period, count, x, note] 
                print(data)
                result.append(data)

            except:
                pass
                
            # 처음 페이지로 이동
            driver.find_element(By.ID,'목록').click()
            time.sleep(1.2)

            table = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[3]/table')
            tbody = table.find_element(By.TAG_NAME, 'tbody')
            rows = tbody.find_elements(By.TAG_NAME, 'tr')
            
        except:
            pass

    try:
        pages = driver.find_element(By.XPATH, '//*[@id="b033091"]/div[2]/div[1]')
        pages.find_element(By.ID, 'pageinput'+str(i+1)).click()
        time.sleep(1)

        if (i+1)%10 == 0:
            driver.find_element(By.XPATH, '//*[@id="nextinput"]/span/button').click()
            time.sleep(1)
    except:
        break

outer:   0%|          | 0/87 [00:00<?, ?it/s]

inner:   0%|          | 0/11 [00:00<?, ?it/s]

['833', 'KB 미국 대표성장주 펀드 가입 고객 EVENT', '2020.05.25 ~ 2020.06.30', '1906', ['https://oimg1.kbstar.com/img/oevent/2020/0525_kbfund/event_visual.jpg', 'https://oimg1.kbstar.com/img/oevent/2020/0525_kbfund/event_cont_01.jpg', 'https://oimg1.kbstar.com/img/oevent/2020/0525_kbfund/event_cont_02.jpg', 'https://oimg1.kbstar.com/img/oevent/2020/0525_kbfund/event_cont_03.jpg'], '알아두세요\n집합투자상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.\n집합투자상품은 운용결과에 따라 투자원금의 손실이 발생할 수 있으며 그 손실은 투자자에게 귀속됩니다.\n집합투자증권을 취득하시기 전에 투자대상, 보수·수수료 및 환매방법 등에 관하여 (간이)투자설명서를 반드시 읽어보시기 바랍니다.\n해외의 증권에 투자하는 펀드는 환율변동 및 투자대상 국가의 시장, 정치 및 경제상황 등에 따른 위험으로 원금 손실이 발생할 수 있습니다.\n본 이벤트는 KB국민은행, KB자산운용 및 제휴업체의 사정에 따라 변경 또는 중단될 수 있습니다. (기 조건 충족 및 당첨고객 제외)\n당첨자 발표일 이후 한 달 이내 경품 수령거부 또는 연락처 오류 등의 사유로 연락불가시 당첨이 무효처리 됩니다. 내 고객정보 바로가기 >\n경품제공을 위해 수탁업체에게 고객님의 정보를 제공하고 경품 발송 업무를 위탁합니다. (수탁업체: ㈜코원비전, 제공범위: 전화번호, 대상업무 : 경품지급)\n기타 자세한 상품 및 이벤트 내용은 창구 직원 또는 스마트상담부(☎1588-9999)로 문의하시거나 KB국민은행 홈페이지 (www.kbstar.com)를 참조하시기 바랍니다.\n본 광고물에 대한 유효기간 2020.06.30까지준법감시인 심의필

inner:   0%|          | 0/11 [00:00<?, ?it/s]

inner:   0%|          | 0/11 [00:00<?, ?it/s]

...

inner:   0%|          | 0/11 [00:00<?, ?it/s]

inner:   0%|          | 0/11 [00:00<?, ?it/s]

inner:   0%|          | 0/11 [00:00<?, ?it/s]

inner:   0%|          | 0/11 [00:00<?, ?it/s]

inner:   0%|          | 0/11 [00:00<?, ?it/s]

In [31]:
columns = ['번호','제목','이벤트 기간','조회수','img','알아두세요']
kb_event = pd.DataFrame(result, columns=columns)
kb_event

,번호,제목,이벤트 기간,조회수,img,알아두세요
0,833,KB 미국 대표성장주 펀드 가입 고객 EVENT,2020.05.25 ~ 2020.06.30,1906,[https://oimg1.kbstar.com/img/oevent/2020/0525...,알아두세요\n집합투자상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.\n...
1,837,삼성전자 펀드 투자하고 선물받자,2020.05.13 ~ 2020.06.30,1547,[https://oimg1.kbstar.com/img/oevent/2020/0612...,알아두세요\n집합투자상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.\n...


In [28]:
len(kb_event)

55

In [29]:
import openpyxl
kb_event.to_excel('/Users/parkjungeun/Downloads/kb_event8.xlsx', index=False)

In [16]:
img = kb_event.loc[71,'img']
img

['https://oimg1.kbstar.com/img/oevent/2023/0823_Liiv&Live/ev_visual.png',
 'https://oimg1.kbstar.com/img/oevent/2023/0823_Liiv&Live/ev_cont_01.png',
 'https://oimg1.kbstar.com/img/oevent/2023/0823_Liiv&Live/ev_cont_02.png',
 'https://oimg1.kbstar.com/img/oevent/2023/0823_Liiv&Live/ev_cont_03.png']

### 이미지 url -> 텍스트

In [25]:
#! pip3 install pytesseract
from urllib import request
from PIL import Image
import pytesseract
from io import BytesIO

res = request.urlopen(img[0]).read()
img = Image.open(BytesIO(res))
print(img.size)
#print(img.show())

(720, 340)


In [26]:
#text = pytesseract.image_to_string(img, lang = 'Hangul')
text = pytesseract.image_to_string(img, lang = 'eng+kor')
print(text)

>kb 1<8 콕 민 은 명 * Knitted,
한정판 다 미 머리 Kit
2024 개

무 료 증 정

‘S

